[View in Colaboratory](https://colab.research.google.com/github/Sarkin/nlp2018/blob/master/Copy_of_Lecture_1_Keras.ipynb)

# Введение в keras

In [0]:
!pip install -q keras

## Sequential Model

Обучить свою нейросеть просто! Нужно

1.   Выбрать типы и последовательность слоев
2.   Настроить параметры слоев (передав нужные в конструктор слоя или оставив вариант по умолчанию)
3.   Выбрать оптимизатор и скомпилировать модель
4.   Обучить модель!


Чтобы создать Sequential модель, просто позовите следующие строки:

In [3]:
from keras.models import Sequential

model = Sequential()

Using TensorFlow backend.


Слои теперь добавляются в модель последовательно, как будто это list:

In [0]:
from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

Только что мы создали сеть с одним скрытым слоем. Сколько в ней обучаемых параметров?

Посмотреть это можно с помощью метода

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 7,114
Trainable params: 7,114
Non-trainable params: 0
_________________________________________________________________


*Почему столько параметров в слоях?* (input_dim x units) + (units bias весов)

*Что за `None` вместо первого элемента `Output Shape`?*  Переменное количество входных векторов, по сути batch size

Прежде чем обучать эту модель, её нужно скомпилировать:

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

В NLP чаще всего ставятся задачи классификации, поэтому нужно запомнить такие функции потерь:


*   **categorical_crossentropy** - для многоклассовой классификации, в качестве меток должны передаваться one-hot-encoding вектора
*   **sparse_categorical_crossentropy** - аналогично предыдущему, но в качестве меток нужно передавать просто индексы соответствующих классов
*   **binary_crossentropy** - для бинарной классификации


В качестве оптимизатора обычно используют `sgd` или `adam`.


In [0]:
import numpy as np

X_train = np.random.randn(10000, 100)
y_train = np.random.randint(0, 10, size=10000)
X_test = np.random.randn(1000, 100)

In [8]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
10000/10000 [==============================] - 3s 258us/step - loss: 2.5663 - acc: 0.1026
Epoch 2/5
10000/10000 [==============================] - 1s 150us/step - loss: 2.4418 - acc: 0.1088
Epoch 3/5
10000/10000 [==============================] - 2s 156us/step - loss: 2.3806 - acc: 0.1143
Epoch 4/5
10000/10000 [==============================] - 1s 149us/step - loss: 2.3431 - acc: 0.1218
Epoch 5/5
10000/10000 [==============================] - 2s 152us/step - loss: 2.3177 - acc: 0.1301


## Binary Classification

Попробуем предсказать сентимент (положительность/отрицательность) imdb'шных ревью [keras: IMDB Movie reviews sentiment classification](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)

In [0]:
from keras.datasets import imdb

In [10]:
NUM_WORDS = 10000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
17465344/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences


### Bag-of-words

Начнем с простейшей модели

In [0]:
def convert_to_bow(X):
  X_bow = np.zeros((len(X), NUM_WORDS))
  for i, review in enumerate(X):
    for ind in review:
      X_bow[i, ind] = 1
  return X_bow

X_train_bow, X_test_bow = convert_to_bow(X_train), convert_to_bow(X_test)

In [12]:
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=NUM_WORDS))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 10001     
Total params: 10,001
Trainable params: 10,001
Non-trainable params: 0
_________________________________________________________________


*Как называется такая модель?* softmax classifier

In [13]:
model.fit(X_train_bow, y_train, 
          batch_size=32,
          epochs=3,
          validation_data=(X_test_bow, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 8s 336us/step - loss: 0.4362 - acc: 0.8456 - val_loss: 0.3476 - val_acc: 0.8807
Epoch 2/3
25000/25000 [==============================] - 8s 327us/step - loss: 0.2861 - acc: 0.9035 - val_loss: 0.3035 - val_acc: 0.8874
Epoch 3/3
25000/25000 [==============================] - 8s 326us/step - loss: 0.2396 - acc: 0.9201 - val_loss: 0.2894 - val_acc: 0.8888


### Convs

Переходим к более сложным моделям.

In [0]:
from keras.preprocessing import sequence

MAX_LEN = 400

X_train_long = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test_long = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

In [15]:
from keras.layers import Embedding, Dropout, SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.optimizers import Adam

EMB_DIM = 50

model = Sequential()

model.add(Embedding(input_dim=NUM_WORDS, 
                    output_dim=EMB_DIM, 
                    input_length=MAX_LEN))

model.add(Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(),  # оптимизатор ещё и так можно передавать
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           500000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 128)          19328     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 527,649
Trainable params: 527,649
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train_long, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(X_test_long, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 14s 563us/step - loss: 0.4000 - acc: 0.8077 - val_loss: 0.2716 - val_acc: 0.8880
Epoch 2/5
25000/25000 [==============================] - 13s 527us/step - loss: 0.1969 - acc: 0.9251 - val_loss: 0.2629 - val_acc: 0.8902
Epoch 3/5
 5152/25000 [=====>........................] - ETA: 8s - loss: 0.0900 - acc: 0.9730

25000/25000 [==============================] - 13s 528us/step - loss: 0.0870 - acc: 0.9724 - val_loss: 0.3046 - val_acc: 0.8892
Epoch 4/5
25000/25000 [==============================] - 13s 524us/step - loss: 0.0279 - acc: 0.9929 - val_loss: 0.3830 - val_acc: 0.8846
Epoch 5/5
15776/25000 [=================>............] - ETA: 3s - loss: 0.0068 - acc: 0.9989

25000/25000 [==============================] - 13s 528us/step - loss: 0.0068 - acc: 0.9986 - val_loss: 0.4446 - val_acc: 0.8880


Попробуем улучшить качество.

Добавим предобученные словные эмбеддинги.

Чёрная магия, которую нужно для этого скастовать, выглядит как-нибудь так:

In [17]:
!pip install chakin

import chakin
chakin.search(lang='English')

chakin.download(number=11, save_dir='./')

!unzip glove.6B.zip

!pip install gensim
!python -m gensim.scripts.glove2word2vec --input glove.6B.50d.txt --output glove.6B.50d.v2w.txt

  Running setup.py bdist_wheel for chakin ... - done
  Stored in directory: /content/.cache/pip/wheels/0f/00/a8/306996e292c7a68d22e81d6350dc8adbb101b05163eb3b6915
Successfully built chakin
                   Name  Dimension                     Corpus VocabularySize  \
2          fastText(en)        300                  Wikipedia           2.5M   
11         GloVe.6B.50d         50  Wikipedia+Gigaword 5 (6B)           400K   
12        GloVe.6B.100d        100  Wikipedia+Gigaword 5 (6B)           400K   
13        GloVe.6B.200d        200  Wikipedia+Gigaword 5 (6B)           400K   
14        GloVe.6B.300d        300  Wikipedia+Gigaword 5 (6B)           400K   
15       GloVe.42B.300d        300          Common Crawl(42B)           1.9M   
16      GloVe.840B.300d        300         Common Crawl(840B)           2.2M   
17    GloVe.Twitter.25d         25               Twitter(27B)           1.2M   
18    GloVe.Twitter.50d         50               Twitter(27B)           1.2M   
19   GloV

Test: 100% ||                                      | Time:  0:00:13  59.7 MiB/s


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
    100% |████████████████████████████████| 22.6MB 1.5MB/s 
    100% |████████████████████████████████| 1.4MB 14.3MB/s 
    100% |████████████████████████████████| 133kB 13.5MB/s 
    100% |████████████████████████████████| 4.3MB 7.1MB/s 
    100% |████████████████████████████████| 61kB 16.8MB/s 
    100% |████████████████████████████████| 552kB 21.3MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in 

2018-06-18 19:58:11,854 - glove2word2vec - INFO - converting 400000 vectors from glove.6B.50d.txt to glove.6B.50d.v2w.txt
2018-06-18 19:58:12,713 - glove2word2vec - INFO - Converted model with 400000 vectors and 50 dimensions


In [18]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('glove.6B.50d.v2w.txt', binary=False)

glove_word2index = {w : i for i, w in enumerate(word_vectors.index2word)}

word_index = imdb.get_word_index()
word_index_rev = {word_index[x] : x for x in word_index}

embedding_matrix = 0.05 * np.random.randn(NUM_WORDS, EMB_DIM)

num_of_known_words = 0
for word in word_index:
  ind = word_index[word] + 3
  if ind < NUM_WORDS and word in glove_word2index:
    embedding_matrix[ind] = word_vectors.vectors[glove_word2index[word]]
    num_of_known_words += 1

print('Know', num_of_known_words, 'out of', NUM_WORDS)

1646592/1641221 [==============================] - 1s 0us/step
Know 9793 out of 10000


**Задание**: Постройте ту же самую модель, но уже с передачей весов в `Embedding`.

Обратите внимание на параметр `trainable`: во многих случаях дообучать эмбеддинги не нужно - для этого нужно передать `trainable=False`.

In [19]:
model = Sequential()


model.add(Embedding(input_dim=NUM_WORDS, 
                    output_dim=EMB_DIM, 
                    input_length=MAX_LEN,
                    weights=[embedding_matrix]))

model.add(Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 50)           500000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 398, 128)          19328     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 527,649
Trainable params: 527,649
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(X_train_long, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(X_test_long, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 13s 539us/step - loss: 0.4362 - acc: 0.7908 - val_loss: 0.3113 - val_acc: 0.8650
Epoch 2/5
25000/25000 [==============================] - 13s 526us/step - loss: 0.2630 - acc: 0.8928 - val_loss: 0.2782 - val_acc: 0.8846
Epoch 3/5
 4896/25000 [====>.........................] - ETA: 8s - loss: 0.1738 - acc: 0.9310

25000/25000 [==============================] - 13s 539us/step - loss: 0.1736 - acc: 0.9338 - val_loss: 0.2811 - val_acc: 0.8856
Epoch 4/5
25000/25000 [==============================] - 14s 543us/step - loss: 0.0999 - acc: 0.9648 - val_loss: 0.3616 - val_acc: 0.8749
Epoch 5/5
14688/25000 [================>.............] - ETA: 4s - loss: 0.0426 - acc: 0.9879

25000/25000 [==============================] - 13s 530us/step - loss: 0.0434 - acc: 0.9870 - val_loss: 0.3876 - val_acc: 0.8812


### LSTM

Попробуем теперь использовать рекуррентную сеть.

Укоротим немного предложения для скорости:

In [0]:
MAX_LEN = 80

X_train_short = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test_short = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

**Задание**: постройте модель с LSTM.
Первым должен так и идти слой эмбеддингов, а LSTM - применяться поверх него. Результатом будет последнее предсказание LSTM - с учётом всего контекста.

Интересные параметры LSTM:
`LSTM(units=?, dropout=0.0, recurrent_dropout=0.0, return_sequences=False)`

Попробуйте `units=64` и дропауты в районе 0.2.


In [22]:
from keras.layers import LSTM, Bidirectional

model = Sequential()


model.add(Embedding(input_dim=NUM_WORDS, 
                    output_dim=EMB_DIM, 
                    input_length=MAX_LEN,
                    weights=[embedding_matrix],
                    trainable=False))

model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 80, 50)            500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 529,505
Trainable params: 29,505
Non-trainable params: 500,000
_________________________________________________________________


In [23]:
model.fit(X_train_short, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_test_short, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
21920/25000 [=========================>....] - ETA: 23s - loss: 0.6218 - acc: 0.6515

25000/25000 [==============================] - 241s 10ms/step - loss: 0.6131 - acc: 0.6601 - val_loss: 0.5249 - val_acc: 0.7322
Epoch 2/15
13088/25000 [==============>...............] - ETA: 1:29 - loss: 0.5286 - acc: 0.7352

25000/25000 [==============================] - 241s 10ms/step - loss: 0.5156 - acc: 0.7438 - val_loss: 0.4689 - val_acc: 0.7740
Epoch 3/15
 9728/25000 [==========>...................] - ETA: 2:06 - loss: 0.4902 - acc: 0.7606

25000/25000 [==============================] - 248s 10ms/step - loss: 0.4816 - acc: 0.7656 - val_loss: 0.4421 - val_acc: 0.7892
Epoch 4/15
 8480/25000 [=========>....................] - ETA: 2:00 - loss: 0.4701 - acc: 0.7763

25000/25000 [==============================] - 229s 9ms/step - loss: 0.4684 - acc: 0.7738 - val_loss: 0.4306 - val_acc: 0.7962
Epoch 5/15
 8032/25000 [========>.....................] - ETA: 2:04 - loss: 0.4541 - acc: 0.7820

...

25000/25000 [==============================] - 232s 9ms/step - loss: 0.3905 - acc: 0.8215 - val_loss: 0.3778 - val_acc: 0.8255
Epoch 12/15
 7712/25000 [========>.....................] - ETA: 2:08 - loss: 0.3824 - acc: 0.8222

25000/25000 [==============================] - 237s 9ms/step - loss: 0.3808 - acc: 0.8258 - val_loss: 0.3799 - val_acc: 0.8266
Epoch 13/15
 7712/25000 [========>.....................] - ETA: 2:11 - loss: 0.3695 - acc: 0.8349

25000/25000 [==============================] - 239s 10ms/step - loss: 0.3744 - acc: 0.8290 - val_loss: 0.3715 - val_acc: 0.8283
Epoch 14/15
 7680/25000 [========>.....................] - ETA: 2:09 - loss: 0.3674 - acc: 0.8350

25000/25000 [==============================] - 234s 9ms/step - loss: 0.3719 - acc: 0.8318 - val_loss: 0.4142 - val_acc: 0.8094
Epoch 15/15
 7680/25000 [========>.....................] - ETA: 2:09 - loss: 0.3645 - acc: 0.8314

25000/25000 [==============================] - 233s 9ms/step - loss: 0.3640 - acc: 0.8328 - val_loss: 0.3739 - val_acc: 0.8280


### LSTM-CNN

Вообще говоря, LSTM выдает не одно состояние, а много (внимание на `return_sequences`). Почему бы не попробовать использовать их все?

**Задание**: реализуйте свертки и GlobalMaxPooling поверх выхода LSTM.

In [24]:
model = Sequential()


model.add(Embedding(input_dim=NUM_WORDS, 
                    output_dim=EMB_DIM, 
                    input_length=MAX_LEN,
                    weights=[embedding_matrix],
                    trainable=False))

model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

model.add(Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 80, 50)            500000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 80, 64)            29440     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 78, 128)           24704     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total para

In [25]:
model.fit(X_train_short, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_test_short, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
21920/25000 [=========================>....] - ETA: 23s - loss: 0.5715 - acc: 0.6925

25000/25000 [==============================] - 239s 10ms/step - loss: 0.5637 - acc: 0.6993 - val_loss: 0.4852 - val_acc: 0.7613
Epoch 2/15
13088/25000 [==============>...............] - ETA: 1:29 - loss: 0.4828 - acc: 0.7628

25000/25000 [==============================] - 237s 9ms/step - loss: 0.4833 - acc: 0.7649 - val_loss: 0.4453 - val_acc: 0.7886
Epoch 3/15
 9792/25000 [==========>...................] - ETA: 1:56 - loss: 0.4530 - acc: 0.7804

25000/25000 [==============================] - 240s 10ms/step - loss: 0.4493 - acc: 0.7865 - val_loss: 0.4212 - val_acc: 0.8034
Epoch 4/15
 8480/25000 [=========>....................] - ETA: 2:07 - loss: 0.4269 - acc: 0.7969

25000/25000 [==============================] - 245s 10ms/step - loss: 0.4239 - acc: 0.8002 - val_loss: 0.4287 - val_acc: 0.7972
Epoch 5/15
 8000/25000 [========>.....................] - ETA: 2:09 - loss: 0.4108 - acc: 0.8099

...

25000/25000 [==============================] - 240s 10ms/step - loss: 0.3116 - acc: 0.8643 - val_loss: 0.3599 - val_acc: 0.8383
Epoch 12/15
 7648/25000 [========>.....................] - ETA: 2:18 - loss: 0.2919 - acc: 0.8737

25000/25000 [==============================] - 250s 10ms/step - loss: 0.3063 - acc: 0.8656 - val_loss: 0.3698 - val_acc: 0.8395
Epoch 13/15
 7648/25000 [========>.....................] - ETA: 2:10 - loss: 0.2816 - acc: 0.8802

25000/25000 [==============================] - 234s 9ms/step - loss: 0.2885 - acc: 0.8753 - val_loss: 0.3963 - val_acc: 0.8276
Epoch 14/15
 7680/25000 [========>.....................] - ETA: 2:08 - loss: 0.2743 - acc: 0.8829

25000/25000 [==============================] - 234s 9ms/step - loss: 0.2812 - acc: 0.8791 - val_loss: 0.3657 - val_acc: 0.8360
Epoch 15/15
 7680/25000 [========>.....................] - ETA: 2:09 - loss: 0.2681 - acc: 0.8848

25000/25000 [==============================] - 234s 9ms/step - loss: 0.2708 - acc: 0.8828 - val_loss: 0.3769 - val_acc: 0.8413


## Functional API

Альтернативный (и более гибкий) вариант построения модели.

Говоря по-сложному: у каждого объекта типа Layer переопределен метод `__call__`: его можно вызывать и передавать некоторый входной тензор. Возвращаемое значение - результат применения трансформации, задаваемой этим слоем, к данному входу (опять же тензор).

А если по-простому - давайте построим ту же самую бессмысленную модель из самого начала ноутбука, но уже с новым апи.

In [0]:
from keras.models import Model
from keras.layers import Input

In [27]:
inputs = Input(shape=(100,))

hidden_layer = Dense(units=64, activation='relu')(inputs)
outputs = Dense(units=10, activation='softmax')(hidden_layer)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                6464      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                650       
Total params: 7,114
Trainable params: 7,114
Non-trainable params: 0
_________________________________________________________________


**Задание**: сделайте модель, которая применяет свертки с шириной окна 2 и 3 к imdb dataset'у.

Пригодится слой `concatenate` для объединения результатов разных типов сверток.

In [28]:
from keras.layers import concatenate

inputs = Input(shape=(MAX_LEN,))

embedding = Embedding(input_dim=NUM_WORDS, output_dim=EMB_DIM)(inputs)
conv1 = Conv1D(filters=64, kernel_size=2, padding='valid', activation='relu', strides=1)(embedding)
conv2 = Conv1D(filters=64, kernel_size=3, padding='valid', activation='relu', strides=1)(embedding)
g1 = GlobalMaxPooling1D()(conv1)
g2 = GlobalMaxPooling1D()(conv2)
conc = concatenate([g1, g2], axis=1)

dense = Dropout(0.2)(Dense(64, activation='relu')(conc))
classes = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=inputs, outputs=classes)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(),  # оптимизатор ещё и так можно передавать
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 80)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 80, 50)       500000      input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 79, 64)       6464        embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 78, 64)       9664        embedding_5[0][0]                
__________________________________________________________________________________________________
global_max

In [29]:
model.fit(X_train_short, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_test_short, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 13s 523us/step - loss: 0.4589 - acc: 0.7705 - val_loss: 0.3550 - val_acc: 0.8405
Epoch 2/15
25000/25000 [==============================] - 12s 479us/step - loss: 0.2626 - acc: 0.8936 - val_loss: 0.3476 - val_acc: 0.8482
Epoch 3/15
 8160/25000 [========>.....................] - ETA: 6s - loss: 0.1210 - acc: 0.9613

25000/25000 [==============================] - 12s 478us/step - loss: 0.1331 - acc: 0.9544 - val_loss: 0.3960 - val_acc: 0.8430
Epoch 4/15
25000/25000 [==============================] - 12s 478us/step - loss: 0.0443 - acc: 0.9878 - val_loss: 0.5678 - val_acc: 0.8337
Epoch 5/15
20320/25000 [=======================>......] - ETA: 1s - loss: 0.0126 - acc: 0.9976

25000/25000 [==============================] - 12s 475us/step - loss: 0.0129 - acc: 0.9975 - val_loss: 0.6869 - val_acc: 0.8352
Epoch 6/15
25000/25000 [==============================] - 12s 476us/step - loss: 0.0039 - acc: 0.9995 - val_loss: 0.7606 - val_acc: 0.8309
Epoch 7/15
22528/25000 [==========================>...] - ETA: 1s - loss: 9.0024e-04 - acc: 1.0000

25000/25000 [==============================] - 12s 489us/step - loss: 8.6234e-04 - acc: 1.0000 - val_loss: 0.7734 - val_acc: 0.8436
Epoch 8/15
25000/25000 [==============================] - 12s 492us/step - loss: 2.5330e-04 - acc: 1.0000 - val_loss: 0.8114 - val_acc: 0.8442
Epoch 9/15
21056/25000 [========================>.....] - ETA: 1s - loss: 1.4467e-04 - acc: 1.0000

25000/25000 [==============================] - 12s 491us/step - loss: 1.4067e-04 - acc: 1.0000 - val_loss: 0.8489 - val_acc: 0.8437
Epoch 10/15
25000/25000 [==============================] - 12s 492us/step - loss: 8.3572e-05 - acc: 1.0000 - val_loss: 0.8856 - val_acc: 0.8435
Epoch 11/15
20192/25000 [=======================>......] - ETA: 1s - loss: 5.5246e-05 - acc: 1.0000

25000/25000 [==============================] - 12s 492us/step - loss: 5.5999e-05 - acc: 1.0000 - val_loss: 0.9136 - val_acc: 0.8431
Epoch 12/15
25000/25000 [==============================] - 12s 483us/step - loss: 4.7265e-05 - acc: 1.0000 - val_loss: 0.9541 - val_acc: 0.8424
Epoch 13/15
21376/25000 [========================>.....] - ETA: 1s - loss: 2.4897e-05 - acc: 1.0000

25000/25000 [==============================] - 12s 484us/step - loss: 2.4034e-05 - acc: 1.0000 - val_loss: 0.9925 - val_acc: 0.8437
Epoch 14/15
25000/25000 [==============================] - 12s 488us/step - loss: 1.7694e-05 - acc: 1.0000 - val_loss: 1.0231 - val_acc: 0.8441
Epoch 15/15
20704/25000 [=======================>......] - ETA: 1s - loss: 1.9875e-05 - acc: 1.0000

25000/25000 [==============================] - 12s 488us/step - loss: 1.9293e-05 - acc: 1.0000 - val_loss: 1.0723 - val_acc: 0.8421


## Multiclass Classification

Переходим к многоклассовой классификации: [keras: Reuters newswire topics classification](https://keras.io/datasets/#reuters-newswire-topics-classification).

У нас тут 11,228 новостей размеченных по 46 топикам.

In [30]:
from keras.datasets import reuters

NUM_WORDS = 10000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=NUM_WORDS,
                                                         test_split=0.2)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Mean train example len:', np.mean([len(x) for x in X_train]))
print('Mean test example len:', np.mean([len(x) for x in X_test]))

Loading data...
2113536/2110848 [==============================] - 1s 0us/step
8982 train sequences
2246 test sequences
46 classes
Mean train example len: 145.5398574927633
Mean test example len: 147.66117542297417


In [0]:
MAX_LEN = 150

X_train = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

**Задание**: Попробуйте обучить собственную сеть на этих данных.

Обратите внимание, что теперь уже многоклассовая классификация (вспоминаем про `sparse_categorical_crossentropy` и выходной слой с числом unit'ов, равным числу классов, и `softmax` активацией).

In [32]:
model = Sequential()

print(MAX_LEN)

model.add(Embedding(input_dim=NUM_WORDS, 
                    output_dim=EMB_DIM, 
                    input_length=MAX_LEN,
                    weights=[embedding_matrix]))

model.add(Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

150
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 150, 50)           500000    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 148, 128)          19328     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 46)                2990      
Total params: 530,574
Trainable params: 530,574
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/15
8982/8982 [==============================] - 5s 532us/step - loss: 2.1879 - acc: 0.4488 - val_loss: 1.8589 - val_acc: 0.5205
Epoch 2/15
8982/8982 [==============================] - 3s 327us/step - loss: 1.7313 - acc: 0.5699 - val_loss: 1.6501 - val_acc: 0.6033
Epoch 3/15
8982/8982 [==============================] - 3s 328us/step - loss: 1.4575 - acc: 0.6497 - val_loss: 1.4858 - val_acc: 0.6451
Epoch 4/15
8982/8982 [==============================] - 3s 331us/step - loss: 1.2482 - acc: 0.6931 - val_loss: 1.3638 - val_acc: 0.6692
Epoch 5/15
7232/8982 [=======================>......] - ETA: 0s - loss: 1.1026 - acc: 0.7276

8982/8982 [==============================] - 3s 336us/step - loss: 1.0939 - acc: 0.7269 - val_loss: 1.2940 - val_acc: 0.6910
Epoch 6/15
8982/8982 [==============================] - 3s 341us/step - loss: 0.9374 - acc: 0.7650 - val_loss: 1.2542 - val_acc: 0.6995
Epoch 7/15
8982/8982 [==============================] - 3s 345us/step - loss: 0.7901 - acc: 0.7996 - val_loss: 1.2242 - val_acc: 0.7115
Epoch 8/15
8982/8982 [==============================] - 3s 329us/step - loss: 0.6624 - acc: 0.8250 - val_loss: 1.1966 - val_acc: 0.7186
Epoch 9/15
8982/8982 [==============================] - 3s 330us/step - loss: 0.5716 - acc: 0.8491 - val_loss: 1.2321 - val_acc: 0.7235
Epoch 10/15
2176/8982 [======>.......................] - ETA: 2s - loss: 0.4890 - acc: 0.8644

8982/8982 [==============================] - 3s 326us/step - loss: 0.4857 - acc: 0.8662 - val_loss: 1.2248 - val_acc: 0.7329
Epoch 11/15
8982/8982 [==============================] - 3s 329us/step - loss: 0.4109 - acc: 0.8874 - val_loss: 1.2267 - val_acc: 0.7289
Epoch 12/15
8982/8982 [==============================] - 3s 330us/step - loss: 0.3612 - acc: 0.9024 - val_loss: 1.3076 - val_acc: 0.7275
Epoch 13/15
8982/8982 [==============================] - 3s 334us/step - loss: 0.3281 - acc: 0.9100 - val_loss: 1.3434 - val_acc: 0.7231
Epoch 14/15
8982/8982 [==============================] - 3s 332us/step - loss: 0.2962 - acc: 0.9183 - val_loss: 1.3335 - val_acc: 0.7342
Epoch 15/15
1888/8982 [=====>........................] - ETA: 2s - loss: 0.2496 - acc: 0.9253

8982/8982 [==============================] - 3s 328us/step - loss: 0.2767 - acc: 0.9224 - val_loss: 1.3886 - val_acc: 0.7386


## Классификация на уровне символов

Попробуем вернуться к imdb и предсказывать слова, используя их символьное представление.

In [34]:
NUM_WORDS = 50000

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

word_index = imdb.get_word_index()
word_index_rev = {word_index[x] : x for x in word_index}

Loading data...
25000 train sequences
25000 test sequences


Зададим отображение из символов в индексы.

In [0]:
from string import punctuation

def get_range(first_symb, last_symb):
  return set(chr(c) for c in range(ord(first_symb), ord(last_symb) + 1))

chars = get_range('a', 'z') | get_range('0', '9') | set(punctuation)
char_index = {c : i for i, c in enumerate(chars)}

def get_char_index(char, char_index):
  return char_index[char] if char in char_index else len(char_index)

Используя костыли, построим тензор, в котором на месте каждого элемента стоит последовательность его символов.

In [36]:
MAX_WORD_LEN = 15
MAX_LINE_LEN = 100

X_train = sequence.pad_sequences(X_train, maxlen=MAX_LINE_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_LINE_LEN)

def build_chars_tensor(X):
  X_chars = np.zeros((len(X), MAX_LINE_LEN, MAX_WORD_LEN))
  for i, line in enumerate(X):
    for j, word_ind in enumerate(line):
      if word_ind >= 3:
        word = word_index_rev[word_ind]
        word = word if len(word) < MAX_WORD_LEN else word[-MAX_WORD_LEN:]
        X_chars[i, j, -len(word):] = [get_char_index(c, char_index) for c in word]
  return X_chars

X_chars_train = build_chars_tensor(X_train)
X_chars_test = build_chars_tensor(X_test)

print(X_chars_train.shape)

(25000, 100, 15)


*Какая размерность получится, если применить к этому тензору ещё и слой эмбеддингов?*

**Задание**: допишите код, попробуйте поклассифицировать с помощью него.

In [37]:
from keras.layers import TimeDistributed

def build_chars_layer(chars_count, char_emb_dim=20, lstm_dim=32, dropout_rate=.2):
  chars_embedding = Embedding(chars_count, char_emb_dim, name='char_embeddings')
  chars_lstm = TimeDistributed(Bidirectional(
      LSTM(lstm_dim, dropout=dropout_rate, recurrent_dropout=dropout_rate, name='char_LSTM')))
  
  def process_input(inp):
    res = chars_embedding(inp)
    return chars_lstm(res)
  return process_input
  
chars = Input(shape=(None, MAX_WORD_LEN), name='chars')

chars_level_embedding = build_chars_layer(chars_count = len(char_index) + 1)
chars_output = chars_level_embedding(chars)

lstm = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(chars_output)
outputs = Dense(1, activation='sigmoid')(lstm)


model = Model(inputs=chars, outputs=outputs)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
chars (InputLayer)           (None, None, 15)          0         
_________________________________________________________________
char_embeddings (Embedding)  (None, None, 15, 20)      1380      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 64)          13568     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 48,037
Trainable params: 48,037
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(X_chars_train, y_train,
          batch_size=32,
          epochs=15,
          validation_data=(X_chars_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
21888/25000 [=========================>....] - ETA: 46s - loss: 0.6334 - acc: 0.6251

25000/25000 [==============================] - 448s 18ms/step - loss: 0.6321 - acc: 0.6258 - val_loss: 0.6229 - val_acc: 0.6359
Epoch 2/15
13056/25000 [==============>...............] - ETA: 2:56 - loss: 0.6226 - acc: 0.6416

25000/25000 [==============================] - 452s 18ms/step - loss: 0.6199 - acc: 0.6421 - val_loss: 0.6004 - val_acc: 0.6619
Epoch 3/15
 9696/25000 [==========>...................] - ETA: 3:39 - loss: 0.6135 - acc: 0.6471

25000/25000 [==============================] - 436s 17ms/step - loss: 0.6049 - acc: 0.6566 - val_loss: 0.5919 - val_acc: 0.6742
Epoch 4/15
 8448/25000 [=========>....................] - ETA: 4:01 - loss: 0.6046 - acc: 0.6618

25000/25000 [==============================] - 440s 18ms/step - loss: 0.5977 - acc: 0.6665 - val_loss: 0.5700 - val_acc: 0.6970
Epoch 5/15
 7968/25000 [========>.....................] - ETA: 4:09 - loss: 0.5870 - acc: 0.6845

...

25000/25000 [==============================] - 444s 18ms/step - loss: 0.5173 - acc: 0.7393 - val_loss: 0.4989 - val_acc: 0.7467
Epoch 12/15
 7616/25000 [========>.....................] - ETA: 4:13 - loss: 0.5090 - acc: 0.7413

25000/25000 [==============================] - 443s 18ms/step - loss: 0.5088 - acc: 0.7403 - val_loss: 0.4827 - val_acc: 0.7600
Epoch 13/15
 7616/25000 [========>.....................] - ETA: 4:14 - loss: 0.5057 - acc: 0.7383

25000/25000 [==============================] - 445s 18ms/step - loss: 0.5055 - acc: 0.7431 - val_loss: 0.4780 - val_acc: 0.7650
Epoch 14/15
 7616/25000 [========>.....................] - ETA: 4:13 - loss: 0.5026 - acc: 0.7407

25000/25000 [==============================] - 442s 18ms/step - loss: 0.4929 - acc: 0.7508 - val_loss: 0.4766 - val_acc: 0.7678
Epoch 15/15
 7616/25000 [========>.....................] - ETA: 4:11 - loss: 0.4891 - acc: 0.7576

25000/25000 [==============================] - 441s 18ms/step - loss: 0.4841 - acc: 0.7582 - val_loss: 0.4635 - val_acc: 0.7716
